### Data used: https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [126]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [127]:
with open("../data/shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [128]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [129]:
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### Encoder and Decoder section

#### Further development idea:
Use Sentencepiece (https://github.com/google/sentencepiece) or Tiktoken (https://github.com/openai/tiktoken) intead of the following mapping

In [130]:
stoi = {
    ch:i for i, ch in enumerate(chars)
    }

itos = {
    i:ch for i, ch in enumerate(chars)
    }

In [131]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [132]:
print(encode("Encode this"))
print(decode(encode("Encode this")))

[17, 52, 41, 53, 42, 43, 1, 58, 46, 47, 57]
Encode this


#### Encoding the entire dataset

In [133]:
data = torch.tensor(encode(text), dtype=torch.long)

In [134]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [135]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [136]:
block_size = 8
batch_size = 4
train_data[:block_size + 1]

torch.manual_seed = 9876

In [137]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i+block_size + 1] for i in ix])

    return x, y

In [138]:
xb, yb = get_batch("train")

In [139]:
class BiagramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None): #B, T, C = Batch, Time, Channel
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #Pytorch cross_entropy expects B, T and C in this way when dealing with multi dimensional structures
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [140]:
model = BiagramLanguageModel(vocab_size)
logits, loss = model(xb, yb)

print(logits.shape)

torch.Size([32, 65])


In [141]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


:LHCAIliOCMyO?GNEvQ ?e
,KaKFMY&
xwJqpcSr,cOx:yuVFyx3BqU..bwrLvf,qqiPFYiinl'CErZ?pNzzhSL.XjLy. puIdlG


In [142]:
print(loss)

tensor(4.5780, grad_fn=<NllLossBackward0>)


### Negative log likelyhood

In [143]:
math.log(1/vocab_size) * -1

4.174387269895637

Expecting ~4.17 loss

In [144]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    #Evaluating the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5245184898376465


#### Let's check the improvement

In [145]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


LICENGrs or Wou ay nchidesand tima go wandow w, BathosEWayast atthita soorethofrispl t come ate isur, n
Wh aifung ht O:
CI think ffrde?
Tr.
NG s rof s tl pothe ed.
Hiare; limellathofo'ome o ar wsirdeanoonk' gas le-, wic, cean, ck athe te;
Wh ms hird iucl t.
Bu, to pain t s,
O:


Tirgheve w Yort o am?
Be wastht heal' joat  bondy wil hinke me ty t ut id, atorn.
FO MNRD s lldand

BETI'deasto ak, wie s mpanarishe ve wapry, woway y iatt fod and pFalsstheanh im owan,
TR:
Ed t, yaletesh bu ba che my st


#### No numbers is a good start

In [146]:
B, T, C = 4, 8, 2

x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [147]:
xbow = torch.zeros(B, T, C)

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [149]:
x[0]

tensor([[ 0.0170, -1.1547],
        [ 1.1137, -1.8763],
        [-0.7342,  0.5307],
        [-0.2055,  1.1377],
        [-1.4573, -0.3498],
        [-2.2151, -0.5365],
        [ 0.7738, -0.1415],
        [-1.5323,  0.2518]])

Nested Loops are not efficient, but we can utilise matrix multiplication to achieve more efficient way for self attention (i.e. information about the previous tokens).

#### Toy example

In [156]:
a= torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("________________________________")
print("a==")
print(a)
print("________________________________")
print("b==")
print(b)
print("________________________________")
print("c==")
print(c)
print("________________________________")

________________________________
a==
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
________________________________
b==
tensor([[9., 2.],
        [6., 9.],
        [1., 9.]])
________________________________
c==
tensor([[9.0000, 2.0000],
        [7.5000, 5.5000],
        [5.3333, 6.6667]])
________________________________


#### Toy example ends

## First version

In [165]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x

In [176]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x


In [177]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])